#### 

Goal: Review upcoming weather to determine when would be the best time period for someone to run.   
- Agent 1: Extract longitude and latitude coordinates from city in prompt, return weather 
- Agent 2: Evaluate when is best time period to run

### Import Credentials

In [ ]:
# !pip install openai
# !pip install requests

In [1]:
from creds import credentials
import calendars
import weather
import json
from openai import OpenAI
import os

/Users/jparks/Repositories/tool-calling-OpenAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# get keys
openai_key = credentials['tool-calling-OpenAI']
weather_api = credentials['weather_api']

## *Agent #1: Extract location and get weather*

In [3]:
# 1A: Define function

weather_tool = [{
    # weather tool
        "type": "function",
        "name": "get_weather",
        "description":"get temperature, humidity, precipitation, wind speed for specified location for the next 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "lat": {"type":"number"},
                "long":{"type":"number"}
            },
            # all params in required b/c strict = True
            "required": ["lat", "long"],
            "additionalProperties": False
        },
    # recommended to always use true
    # does not allow model to skip parameteres
    "strict" : True
}]

### *Agent #2: Get Upcoming calendar events*

In [4]:
calendar_get_events = [{
    # weather tool
        "type": "function",
        "name": "get_upcoming_event",
        "description":"Get the days and times from teh user calendar of when they will be busy in the upcoming days. The furthest you can look ahead is 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "cal_id": {"type":"string",
                           "description":"""ID of the calendar to use. It is a random combination of letters and numbers. 
                           Do not use this parameter if they do not give an ID."""},
                "in_future":{"type":"number",
                             "description":"Number of days to look ahead. Do not fill in if the user does not provide it, there is a default value in function."}
            },
            # all params in required when strict = True
            # "required": ["cal_id", "in_future"],
            "additionalProperties": False
        },
    # setting to False
    # so blank string doen't override defaults
    "strict" : False
}]

### *Agent #3: Review the weather and evaluate when it would be best to run outside*

In [15]:
input_message = [{
    "role":"user",
    "content": """I live in Anchorage, Alaska. "
    "What is the best day and time to go for a 30 minute run based on the weather and my work schedule for the next 3 days? "
    "Please schedule it between meetings.""",
}]

In [16]:
client = OpenAI(api_key=openai_key)

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=(calendar_get_events + weather_tool)
)

In [17]:
response.output

[ResponseFunctionToolCall(arguments='{"cal_id":"your_calendar_id_here","in_future":3}', call_id='call_Ui37ZHVYztV5IoqUxM020GBV', name='get_upcoming_event', type='function_call', id='fc_680911c2c9b481928fe1db342ae2f4a70a64fd8ba54a267c', status='completed'),
 ResponseFunctionToolCall(arguments='{"lat":61.2181,"long":-149.9003}', call_id='call_Vft3zV4pKTQ4DEagwnLcZowj', name='get_weather', type='function_call', id='fc_680911c2d6c08192a8199af65d9d10d40a64fd8ba54a267c', status='completed')]

In [18]:
def call_function(name, args):
    if name == "get_weather":
        return weather.get_weather(**args)
    if name == "get_upcoming_event":
        return calendars.get_upcoming_events(**args)

In [19]:
for tool_call in response.output:
    if tool_call.type != "function_call":
        continue
    
    # get function name
    name = tool_call.name
    args = json.loads(tool_call.arguments)
    
    # drop filling string statement of cal_id
    if tool_call.name == 'get_upcoming_event':
        print(args['cal_id'])
        id_has_num = any(char.isdigit() for char in args['cal_id'])
        if not id_has_num: args.pop('cal_id', None)

    # call function + get output
    result = call_function(name, args)
    input_message.append(tool_call)
    input_message.append({
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    })

your_calendar_id_here
Lat: 61.2181, Long: -149.9003


In [20]:
input_message

[{'role': 'user',
  'content': 'I live in Anchorage, Alaska. "\n    "What is the best day and time to go for a 30 minute run based on the weather and my work schedule for the next 3 days? "\n    "Please schedule it between meetings.'},
 ResponseFunctionToolCall(arguments='{"cal_id":"your_calendar_id_here","in_future":3}', call_id='call_Ui37ZHVYztV5IoqUxM020GBV', name='get_upcoming_event', type='function_call', id='fc_680911c2c9b481928fe1db342ae2f4a70a64fd8ba54a267c', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_Ui37ZHVYztV5IoqUxM020GBV',
  'output': "[{'start': '2025-04-23T15:00:00Z', 'end': '2025-04-23T16:00:00Z', 'free_busy_status': 'busy'}, {'start': '2025-04-23T17:30:00Z', 'end': '2025-04-23T18:00:00Z', 'free_busy_status': 'busy'}, {'start': '2025-04-23T18:00:00Z', 'end': '2025-04-23T19:00:00Z', 'free_busy_status': 'busy'}, {'start': '2025-04-23T21:00:00Z', 'end': '2025-04-23T23:30:00Z', 'free_busy_status': 'busy'}, {'start': '2025-04-24T15:00:00Z', 'en

Need to add tool call to input message, along with output. In this scenario below, we are asking the question, the model will request to use one of the tools, we respond you already used it and this was the provided output, so you have the info you need to answer the question and don't actually have to make a tool call.

In [21]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=(calendar_get_events + weather_tool),
)

In [22]:
response.output_text

'Based on your schedule and the weather forecast, the best time for a 30-minute run appears to be on the afternoon of April 24th, between 4:00 PM and 5:00 PM local time. The weather looks favorable with a temperature around 35.6°F, low humidity, no precipitation, and gentle wind. Would you like me to schedule the run for you?'